Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Model Development

In the introductory lab, we explored ways to perform anomaly detection.  We implemented our own solution to labeling data points that were several standard deviations from the mean of the data points.  Then we estimated linear and seasonal trends in the time series, and removed them to perform anomaly detection on the remaining time series.

Here, we are going to apply this approach to our telemetry data.  We will first develop a solution that processes all the data in batch, and then optimize the solution so that it detects anomalies online, as a new sensor measurement arrives.

In this lab, we will do the following:
- Use python port [pyculiarity](https://github.com/nicolasmiller/pyculiarity) of [Twitter's anomaly detection R library](https://github.com/twitter/AnomalyDetection)
- Apply this library to detect anomalies in our telemetry data
- Optimize our approach to perform online anomaly detection

## Prerequisites:

In order to run this lab, you may have to install these additional python packages (unless you already installed them):
- pyculiarity
- numpy

Do you remember from the previous lab how to do this?

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import os
from pyculiarity import detect_ts # python port of Twitter AD lib
import matplotlib.pyplot as plt # for plotting
import seaborn # for decent default settings for figures
import time # so we can time our operations

## Load data

As usual, we start by loading the data

In [ ]:
print("Reading data ... ", end="")
telemetry = pd.read_csv(os.path.join('..', 'data', 'telemetry.csv'))
print("Done.")

print("Parsing datetime...", end="")
telemetry['datetime'] = pd.to_datetime(telemetry['datetime'], format="%m/%d/%Y %I:%M:%S %p")
print("Done.")

## Data Preparation

Let's pick a random sensor (e.g. volt) and machine id and see what we get.

In [ ]:
sensor = 'volt'
machine_id = 67

# Select the slice of the telemetry for this machine 
df_s = telemetry.loc[telemetry.loc[:, 'machineID'] == machine_id, ('datetime', sensor)]
df_s.columns = ['timestamp', 'value']

In [ ]:
# let's take a peak at the data
fig = plt.figure(figsize=(16,4))
plt.plot(df_s['timestamp'], df_s['value'])
df_s.describe()

In [ ]:
results = detect_ts(df_s) # we use the detect_ts function of pyculiarity
anoms = results['anoms']

plt.plot(df_s['timestamp'], df_s['value'])
plt.plot(anoms['timestamp'], anoms['anoms'], 'o')

Hmmm. Looks like we don't really get too many anomalies this way.  We could try to lower our thresholds. Should we try a lower threshold?

In [ ]:
alpha = 0.2 # The level of statistical significance with which to accept or reject anomalies. (default: 0.05)

results = detect_ts(df_s, alpha=alpha)
anoms = results['anoms']

plt.plot(df_s['timestamp'], df_s['value'])
plt.plot(anoms['timestamp'], anoms['anoms'], 'o')

That doesn't really seem too change to much. We could lower the threshold further, but maybe we need to take a completely different approach.

## Denoising data

The data look really noisy. Maybe what we need to do is to calculate running averages, because it is not individual measurements that are a problem, but a *collection* of measurements.

Let's start by aggregating sensor measurements over the last 12h.  Let's use exponential decay, so that more recent measurements inside the time window have more weight that those that are at the beginning of the time window.

In [ ]:
com = 6 # Specify decay in terms of center of mass (this approximates averageing over about twice as many hours)
rm = df_s['value'].ewm(com=com).mean()

plt.plot(df_s['timestamp'], df_s['value'])
plt.plot(df_s['timestamp'], rm)

Interesting, now there seem to be some more apparent anomalies.  Apparently, sometimes sensor readings are off for several hours in a row.  Let's try to run anomaly detection on the smoother time series.

In [ ]:
# we create a data frame with two columns (timestamp, value), which is what pyculiarity expects as input
df_smooth = pd.DataFrame(data={'timestamp': df_s['timestamp'], 'value': rm})

results = detect_ts(df_smooth)
anoms = results['anoms']

# we store the location of the last anomaly for later
last_anomaly = int(np.where(df_smooth['timestamp'] == results['anoms'].tail(1)['timestamp'].values[0])[0])

In [ ]:
plt.plot(df_smooth['timestamp'], df_smooth['value'])
plt.plot(anoms['timestamp'], anoms['anoms'], 'o', alpha=.2)
plt.xlabel("Dates")
plt.ylabel("Sensor Value")

### Hands-on lab

That appears to work, but we want our scoring service to work in real-time.  That means we shouldn't be calculating running averages over all the historic data whenever we receive just one new row of data.  Instead we want to use the last existing value of the running average, integrate it with the new data, to get the new running average.  Put another way, we want to calculate running averages *online*, rather than in *batch*.

### Calculating running averages online

The first step is to create a lightweight function that can calculate running avgs. 

[Welford's online](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Welford's_Online_algorithm) algorithm for calculating running variance includes a method to also calculate the mean.  This method is also described in Donald Knuth's Art of Computer Programming, Vol 2, page 232, 3rd edition.  

In [ ]:
import numpy as np

def run_avg(ts):
    rm_o = np.zeros_like(ts)
    rm_o[0] = -1 # enter your solution here

    for r in range(1, len(rm)):
        curr_com = float(min(com, r))
        rm_o[r] = -1 # enter your solution here
    return rm_o

In [ ]:
# %cat "../solutions/running_avg.py"

 Let's confirm that your solution is correct. If your online version produces the same results as the batch version, you should just see an orange line in the below plot.  That is, a perfect blend of red and yellow. If you also see red and yellow, that means your online solution falls short of implmenting the batch version for calculating running averages.

In [ ]:
rm_o = run_avg(df_s['value'].values)

plt.plot(df_smooth['timestamp'], df_smooth['value'], color='yellow')
plt.plot(df_smooth['timestamp'], rm_o, alpha=.5, color='red')

You can also plot he difference between the two resulting time series.  Except for the couple of first values in the timeseries, this should all be zeros.

In [ ]:
plt.plot(df_smooth['timestamp'], df_smooth['value'] - rm_o)

### End of Lab

## Optimizing the window over which to perform Anomaly Detection

One remaining optimization is to only use the last couple of values of the running average for anomaly detection, instead of running it repeatedely on the entire timeseries of the running average.  There is no general answer for how many recent values have to be included to get robust anomaly detection, because the choice of the size of this sliding window depends on all sorts of factors (e.g. machine, sensor, size of anomaly).  

This means we have to develop an empirical approach to finding the right size for this sliding window.   

The first step is to define a function that performs anomaly detection using only those values that fall within a specified time window leading up to the current value.

The smaller the window, the faster the anomaly detection algorithm runs.  However, if we make the window too small, the algorithm will no longer be able to detect linear and seasonal trends, and will fail.

Below is a function that allows you to test different window sizes for anomaly detection.  It expects a data frame with two columns (timestamp, value), a specification of the window size, and the row index of an anomaly in the provided data frame.  The function returns whether it found an anomaly and how long it took to run (in seconds).

In [ ]:
def eval(df_smooth, window_size, stop):
    is_anomaly = False
    run_time = -1
    start_index = max(0, stop - window_size)
    df_win = df_smooth.iloc[start_index:stop, :]
    try:
        start_time = time.time()
        results = detect_ts(df_win)
        run_time = time.time() - start_time
        if results['anoms'].shape[0] > 0:
            timestamp = df_win['timestamp'].tail(1).values[0]
            if timestamp == results['anoms'].tail(1)['timestamp'].values[0]:
                is_anomaly = True
    except ValueError:
        pass
    return is_anomaly, run_time

let's run this function to test whether we can recover the last anomaly we previously discovered in this time series using the batch method.


In [ ]:
is_anomaly, run_time = eval(df_smooth, 24*7, last_anomaly) 

print('Success: %s, run_time: %ss' % (is_anomaly, round(run_time,2)))

### Hands-on lab

We need to systematically explore how different window sizes affect our ability to detect anomalies in the time series.  When we do this, we need to consider both our precision and sensitivity in doing so.  Depending on the operational cost of false positives and negatives, we may have to use different F-measures.

Start with the following steps:
- Define a function that samples 100 random rows and sensors from the data frame. (you can use `np.random.choice` to select a number of rows and to select a random machine) 
- For each of the samples, you call the `eval` function.
- Record the number of correct and incorrect anomaly detection results, and calculate accuracy measures.

Explore the parameter space:
- Repeat the above for different settings for the size of the sliding window (50, 100, 5000, 1000).
- Create a scatter plot of True positive (x-axis) against False positive (y-axis) rates, with the marker color indicating the size of the sliding window (make sure to include a legend), and the size of the marker indicating the run time.

To conlclude the lab, calculate the cost and savings of different settings for the size of the sliding window.  What if the cost of a false negative is high (e.g. USD 50000 for a broken machine), what if that cost is low (e.g. USD 10 for replacing a broken part and restarting the machine)?  What if a false positive is really expensive (e.g. USD 1000 for production loss while machine is being restarted)?

### End of lab

## Bonus Lab

If you want to speed things up, you can use HyperDrive for this this optimization.  HyperDrive currently supports Batch AI (AML Compute) or DSVM as a compute target. You can get more information on how to do this in this [documenation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters).

Here is a [sample notebook](https://github.com/Azure/MachineLearningNotebooks/tree/master/training/03.train-hyperparameter-tune-deploy-with-tensorflow).

## The end

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.